# Computer Infrastructure

This document will contain solutions/workings for the Computer Infrastructure module, the tasks completed in this are from the [problems.md](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/problems.md) document for this module.
The assessment details can be found in [assessment.md](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/assessment.md) and in the [guidance notebook](https://github.com/ianmcloughlin/computer-infrastructure/blob/main/assessment/guidance.ipynb).